# SQL Database

This notebook showcases an agent designed to interact with a `SQL` databases. 

The agent builds from [SQLDatabaseChain](https://python.langchain.com/docs/use_cases/tabular/sqlite). 

It is designed to answer more general questions about a database, as well as recover from errors.

Note that, as this agent is in active development, all answers might not be correct. 

Additionally, it is not guaranteed that the agent won't perform DML statements on your database given certain questions. 

Be careful running it on sensitive data!|

This uses the example `Chinook` database. 

To set it up follow [these instructions](https://database.guide/2-sample-databases-sqlite/) and place the .db file in a notebooks folder at the root of this repository.

In [15]:
from langchain.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

## Toolkit

In [17]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI

toolkit = SQLDatabaseToolkit(db=db, llm=ChatOpenAI(temperature=0))

In [18]:
tools = toolkit.get_tools()

### Use SQLDatabaseToolkit within an Agent


In [8]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent

In [9]:
# Prompt
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [11]:
# LLM
llm = ChatOpenAI(temperature=0)

In [12]:
# Agent
agent = create_openai_functions_agent(llm, toolkit.get_tools(), prompt)

In [ ]:
# Executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=toolkit.get_tools(),
    verbose=False,
)

## Disclaimer ⚠️

The query chain may generate insert/update/delete queries. When this is not expected, use a custom prompt or create a SQL users without write permissions.

The final user might overload your SQL database by asking a simple question such as "run the biggest query possible". The generated query might look like:

```sql
SELECT * FROM "public"."users"
    JOIN "public"."user_permissions" ON "public"."users".id = "public"."user_permissions".user_id
    JOIN "public"."projects" ON "public"."users".id = "public"."projects".user_id
    JOIN "public"."events" ON "public"."projects".id = "public"."events".project_id;
```

For a transactional SQL database, if one of the table above contains millions of rows, the query might cause trouble to other applications using the same database.

Most datawarehouse oriented databases support user-level quota, for limiting resource usage.

## Example: describing a table

In [19]:
agent_executor.invoke("Describe the schema of the playlisttrack table")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought: I should query the schema of the PlaylistTrack table.
Action: sql_db_schema
Action Input: PlaylistTrack
Observation: 
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought: I should query the schema of the PlaylistTrack table.
Action: sql_db_schema
Action Input: PlaylistTrack
Observation: 
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("P

{'input': 'Describe the schema of the playlisttrack table',
 'output': 'CREATE TABLE "PlaylistTrack" (\n\t"PlaylistId" INTEGER NOT NULL, \n\t"TrackId" INTEGER NOT NULL, \n\tPRIMARY KEY ("PlaylistId", "TrackId"), \n\tFOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), \n\tFOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")\n)\n\n/*\n3 rows from PlaylistTrack table:\nPlaylistId\tTrackId\n1\t3402\n1\t3389\n1\t3390\n*/'}

In [5]:
agent_executor.run("Describe the playlisttrack table")

/Users/rlm/miniforge3/envs/llama2/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought: I should query the schema of the PlaylistTrack table.
Action: sql_db_schema
Action Input: PlaylistTrack
Observation: 
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought: I should query the first 10 rows from the PlaylistTrack table.
Action: sql_db_query
Action Input: SELECT * FROM PlaylistTrack LIMIT 10
Observation: [(1, 3402), (1, 3389), (1, 3390), (1, 3391), (1, 3392), (1, 3393), (1, 3394), (1, 3395), (1, 3396), (1, 3397)]
Thought: I now know the final answer
Final Answer: [(1, 3402), (1,

'[(1, 3402), (1, 3389), (1, 3390), (1, 3391), (1, 3392), (1, 3393), (1, 3394), (1, 3395), (1, 3396), (1, 3397)]'

## Example: describing a table, recovering from an error

In this example, the agent tries to search for a table, `playlists`, that doesn't exist, but finds the next best result

In [29]:
agent_executor.invoke("Describe the playlists table")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought: I should query the schema of the Playlist table.
Action: sql_db_schema
Action Input: Playlist
Observation: 
CREATE TABLE "Playlist" (
	"PlaylistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("PlaylistId")
)

/*
3 rows from Playlist table:
PlaylistId	Name
1	Music
2	Movies
3	TV Shows
*/
Thought: I now know the final answer
Final Answer: The Playlist table contains a PlaylistId column and a Name column. It has 3 rows with the names "Music", "Movies", and "TV Shows".

> Finished chain.


{'input': 'Describe the playlists table',
 'output': 'The Playlist table contains a PlaylistId column and a Name column. It has 3 rows with the names "Music", "Movies", and "TV Shows".'}

## Example: running queries

In [23]:
agent_executor.invoke(
    "List the total sales per country. Which country's customers spent the most?"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought: I should query the Invoice table to get the total sales per country.
Action: sql_db_schema
Action Input: Invoice
Observation: 
CREATE TABLE "Invoice" (
	"InvoiceId" INTEGER NOT NULL, 
	"CustomerId" INTEGER NOT NULL, 
	"InvoiceDate" DATETIME NOT NULL, 
	"BillingAddress" NVARCHAR(70), 
	"BillingCity" NVARCHAR(40), 
	"BillingState" NVARCHAR(40), 
	"BillingCountry" NVARCHAR(40), 
	"BillingPostalCode" NVARCHAR(10), 
	"Total" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("InvoiceId"), 
	FOREIGN KEY("CustomerId") REFERENCES "Customer" ("CustomerId")
)

/*
3 rows from Invoice table:
InvoiceId	CustomerId	InvoiceDate	BillingAddress	BillingCity	BillingState	BillingCountry	BillingPostalCode	Total
1	2	2009-01-01 00:00:00	Theodor-Heuss-Straße 34	Stuttgart	None	Germany	70174	1.98
2	4	2009-01-02 00:00

{'input': "List the total sales per country. Which country's customers spent the most?",
 'output': 'USA'}

In [24]:
agent_executor.invoke(
    "Show the total number of tracks in each playlist. The Playlist name should be included in the result."
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought: I should query the Playlist and PlaylistTrack tables to get the total number of tracks in each playlist.
Action: sql_db_schema
Action Input: Playlist, PlaylistTrack
Observation: 
CREATE TABLE "Playlist" (
	"PlaylistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("PlaylistId")
)

/*
3 rows from Playlist table:
PlaylistId	Name
1	Music
2	Movies
3	TV Shows
*/


CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought: I should use a JOIN statement to combine the two tables and then use C

{'input': 'Show the total number of tracks in each playlist. The Playlist name should be included in the result.',
 'output': "[('TV Shows', 426), ('On-The-Go 1', 1), ('Music Videos', 1), ('Music', 6580), ('Heavy Metal Classic', 26), ('Grunge', 15), ('Classical 101 - The Basics', 25), ('Classical 101 - Next Steps', 25), ('Classical 101 - Deep Cuts', 25), ('Classical', 75), ('Brazilian Music', 39), ('90’s Music', 1477)]"}

## Recovering from an error

In this example, the agent is able to recover from an error after initially trying to access an attribute (`Track.ArtistId`) which doesn't exist.

In [25]:
agent_executor.invoke("Who are the top 3 best selling artists?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought: I should query the Album table to get the artist names and the InvoiceLine table to get the number of sales.
Action: sql_db_schema
Action Input: Album, InvoiceLine
Observation: 
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "InvoiceLine" (
	"InvoiceLineId" INTEGER NOT NULL, 
	"InvoiceId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	"Quantity" INTEGER NOT NULL, 
	PRIMARY KEY ("InvoiceLineId"), 
	FOREIGN KEY("TrackId") REFERENCES

{'input': 'Who are the top 3 best selling artists?',
 'output': 'The top 3 best selling artists are U2, Iron Maiden, and Led Zeppelin.'}